In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../codes/")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../codes/scpy2/")))

from scpy2.utils.nbmagics import install_magics
install_magics()
del install_magics

D:\Program_File\VSCode_Portable\workon_env\pyElan\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
d:\09_Work\Jun_Workspace\Python_Scientific_Computing_Best_Practices\繁中程式碼\codes\scpy2\utils\program_finder.py:25: UserWarning: Inkscape not found
  warnings.warn("Inkscape not found")


In [2]:
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

plt.rcParams["font.family"] = "simsun" # 可以直接修改設定字典，設定預設字型，這樣就不需要在每次繪製文字時設定字型了
plt.rcParams['axes.unicode_minus']=False    # 當座標軸上的 負號無法正常顯示時，需加這行 code

In [3]:
import numpy as np

### 各種乘積運算

表2-9 本節介紹的函數 

| 函數名稱 | 功能 |
|---------|------|
| dot() | 矩陣乘積 |
| inner() | 內積 |
| outer() | 外積 |
| tensordot() | 張量乘積 |

矩陣的乘積可以使用 `dot()` 計算。對於二維陣列，它計算的是矩陣乘積；對於一維陣列，它計算的是內積。當需要將一維陣列當作列向量或行向量進行矩陣運算時，先將一維陣列轉為二維陣列：

In [4]:
a = np.array([1, 2, 3])
%C a[:, None]; a[None, :]

a[:, None]   a[None, :]
----------  -----------
[[1],       [[1, 2, 3]]
 [2],                  
 [3]]                  


對於多維陣列，`dot()` 的通用計算公式如下，即結果陣列中的每個元素都是：陣列 a 的最後軸上的所有元素與陣列 b 的倒數第二軸上的所有元素的乘積和：
```python
dot(a,b)[i,j,k,m] = sum(a[i,j,:]*b[k,:,m])
```
下面以兩個 3D 陣列的乘積示範 `dot()` 的計算結果。首先建立兩個 3D 陣列，這兩個陣列的最後兩軸滿足矩陣乘積的條件：

In [5]:
a = np.arange(12).reshape(2, 3, 2)
b = np.arange(12, 24).reshape(2, 2, 3)
c = np.dot(a, b)
c.shape

(2, 3, 2, 3)

c 是陣列 a 和 b 的多個子矩陣的乘積。我們可以把陣列 a 看作兩個形狀為 (3, 2) 的矩陣，而把矩陣 b 看作兩個形狀為 (2,3) 的矩陣。 a 中的兩個矩陣分別與 b 中的兩個矩陣進行矩陣乘積，就獲得陣列c，`c[i,:,j:]`是 a 中第 i 個矩陣與 b 中第 j 個矩陣的乘積。

In [6]:
for i, j in np.ndindex(2, 2):
    assert np.alltrue( c[i, :, j, :] == np.dot(a[i], b[j]) )

對於兩個一維陣列，`inner()` 和 `dot()` 一樣，計算兩個陣列對應索引元素的乘積和。而對於多維陣列，它計算的結果陣列中的每個元素都是：陣列 a 和 b 的最後軸的內積。因此陣列 a 和 b 的最後軸的長度必須相同：

```python
inner(a,b)[i,j,k,m] = sum(a[i,j,:]*b[k,m,:])
```
下面是對 `inner()` 的示範：

In [7]:
a = np.arange(12).reshape(2, 3, 2)
b = np.arange(12, 24).reshape(2, 3, 2)
c = np.inner(a, b)
c.shape

(2, 3, 2, 3)

In [8]:
for i, j, k, l in np.ndindex(2, 3, 2, 3):
    assert c[i, j, k, l] == np.inner(a[i, j], b[k, l])

`outer()` 只對一維陣列進行計算，如果傳入的是多維陣列，則先將此陣列展平為一維陣列之後再進行運算。它計算列向量和行向量的矩陣乘積：

In [9]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6, 7])
%C np.outer(a, b); np.dot(a[:, None], b[None, :])

  np.outer(a, b)    np.dot(a[:, None], b[None, :])
------------------  ------------------------------
[[ 4,  5,  6,  7],  [[ 4,  5,  6,  7],            
 [ 8, 10, 12, 14],   [ 8, 10, 12, 14],            
 [12, 15, 18, 21]]   [12, 15, 18, 21]]            


`tensordot()` 將兩個多維陣列 a 和 b 指定軸上的對應元素相乘並求和，它是最一般化的乘積運算函數。下面透過一些實例逐步介紹其用法。下面計算兩個矩陣的乘積：  
❶ `axes` 參數有兩個元素，第一個元素表示 a 中的軸，第二個元素表示 b 中的軸，這兩個軸上對應的元素相乘之後求和。  
❷ `axes` 也可以是一個整數，它表示把 a 中的後 `axes` 個軸和 b 中的前 `axes` 個軸進行乘積和運算，而對於乘積和之外的軸則保持不變。

In [10]:
a = np.random.rand(3, 4)
b = np.random.rand(4, 5)

c1 = np.tensordot(a, b, axes=[[1], [0]]) #❶
c2 = np.tensordot(a, b, axes=1)          #❷
c3 = np.dot(a, b)
assert np.allclose(c1, c3)
assert np.allclose(c2, c3)

對於多維陣列的 `dot()` 乘積，可以用 `tensordot(a, b, axes=[[-1],[-2]])` 表示，即將 a 的最後軸和 b 中的倒數第二軸求乘積和：

In [11]:
a = np.arange(12).reshape(2, 3, 2)
b = np.arange(12, 24).reshape(2, 2, 3)
c1 = np.tensordot(a, b, axes=[[-1], [-2]])
c2 = np.dot(a, b)
assert np.alltrue(c1 == c2)

 在下面的實例中，將 a 的第 1, 2 軸與 b 的第 1, 0 軸求乘積和，因此 c 中的每個元素都是按照以下運算式計算的：

 ```python
 c[i,j,k,l] = np.sum(a[i,:,:,j]*b[:,:,k,l].T)
 ```
 注意由於 b 對應的 `axes` 中的軸是倒序的，因此需要做轉置操作。

In [12]:
a = np.random.rand(4, 5, 6, 7)
b = np.random.rand(6, 5, 2, 3)
c = np.tensordot(a, b, axes=[[1, 2], [1, 0]])

for i, j, k, l in np.ndindex(4, 7, 2, 3):
    assert np.allclose(c[i, j, k, l], np.sum(a[i, :, :, j] * b[:, :, k, l].T))
    
c.shape

(4, 7, 2, 3)